In [2]:
# import libaries
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from sklearn.manifold import TSNE

from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from import_classifiers import *
from dataset_classes import *
from settings import *

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

np.random.seed(13)

/home/opontorno/miniconda3/envs/mdlearn/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def plot_features(backbone_name):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    trans = get_trans(model_name=backbone_name)
    testing_dset = make_balanced(mydataset(datasets_path, guidance_path, for_basemodel=False, for_testing=True, transforms=trans))
    testloader = DataLoader(testing_dset, batch_size=1, shuffle=True, num_workers=0, drop_last=False)
    folders = ['bm-dm', 'bm-gan', 'bm-real']
    for folder in folders:
        i = folders.index(folder)
        model = backbone(backbone_name, finetuning=False, as_feature_extractor=True)
        state_dict_path = os.path.join(models_dir, folder, backbone_name+folder[2:]+'.pt')
        model.load_state_dict(torch.load(state_dict_path))
        data, labels = [], []
        for batch in testloader:
            img,label,_= batch
            code = model(img)
            code.to('cpu').detach().numpy()
            labels.append(label.detach().numpy())
            data.append(code)
        data, labels = np.array(data), np.array(labels)
        tsne = TSNE(n_components=2, verbose=0, perplexity=50, n_iter=1000, init='pca')
        tsne_res = tsne.fit_transform(data.squeeze())
        axs[i].scatter(x=tsne_res[:,0], y=tsne_res[:,1], c=labels)
        axs[i].set_title(f'{folder}')

In [4]:
plot_features('densenet161')

Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
/home/opontorno/miniconda3/envs/mdlearn/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/opontorno/miniconda3/envs/mdlearn/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


: 

## Testing robustness

In [4]:
!python testing_complete-models.py --backbone densenet121 --robustness_test True --classification_type binary

Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
-    RAW
96.48%, 94.65%, 98.26%, 96.42%
[[3747   64]
 [ 204 3607]]
-   jpegQF90
95.22%, 93.62%, 96.72%, 95.15%
[[3690  121]
 [ 243 3568]]
-   jpegQF80
92.09%, 93.44%, 90.98%, 92.19%
[[3458  353]
 [ 250 3561]]
-   jpegQF70
87.43%, 94.12%, 83.01%, 88.22%
[[3077  734]
 [ 224 3587]]
-   jpegQF60
83.31%, 95.43%, 76.81%, 85.12%
[[2713 1098]
 [ 174 3637]]
-   jpegQF50
79.02%, 96.38%, 71.54%, 82.12%
[[2350 1461]
 [ 138 3673]]


In [5]:
!python testing_complete-models.py --backbone densenet161 --robustness_test True --classification_type binary

Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
-    RAW
97.60%, 96.67%, 98.50%, 97.58%
[[3755   56]
 [ 127 3684]]
-   jpegQF90
96.35%, 95.78%, 96.89%, 96.33%
[[3694  117]
 [ 161 3650]]
-   jpegQF80
90.57%, 96.85%, 86.04%, 91.12%
[[3212  599]
 [ 120 3691]]
-   jpegQF70
79.13%, 98.01%, 71.14%, 82.44%
[[2296 1515]
 [  76 3735]]
-   jpegQF60
71.71%, 98.64%, 64.11%, 77.71%
[[1707 2104]
 [  52 3759]]
-   jpegQF50
69.38%, 98.98%, 62.17%, 76.37%
[[1516 2295]
 [  39 3772]]


## Testing robustness

In [1]:
!python testing_generalization.py --backbone densenet121 

Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
-   inner_gan
61.50%, 64.02%, 61.70%, 62.84%
[[1134  792]
 [ 717 1276]]
-   outer_gan
72.24%, 63.29%, 77.71%, 69.76%
[[1585  362]
 [ 732 1262]]
-   io_gan
67.88%, 64.93%, 69.91%, 67.33%
[[1360  557]
 [ 699 1294]]
-   inner_dm
68.76%, 63.40%, 70.88%, 66.93%
[[1482  518]
 [ 728 1261]]
-   outer_dm
48.10%, 62.79%, 49.02%, 55.06%
[[ 642 1302]
 [ 742 1252]]
-   io_dm
56.81%, 63.27%, 56.26%, 59.56%
[[ 990  979]
 [ 731 1259]]
-   inner_all
63.27%, 61.84%, 64.22%, 63.00%
[[1261  687]
 [ 761 1233]]
-   outer_all
62.68%, 63.64%, 62.94%, 63.29%
[[1202  746]
 [ 724 1267]]
-   all
63.62%, 64.48%, 64.44%, 64.46%
[[1193  709]
 [ 708 1285]]


In [2]:
!python testing_generalization.py --backbone densenet161

Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/opontorno/.cache/torch/hub/pytorch_vision_v0.10.0
-   inner_gan
65.30%, 67.24%, 65.46%, 66.34%
[[1219  707]
 [ 653 1340]]
-   outer_gan
74.45%, 67.40%, 79.01%, 72.75%
[[1590  357]
 [ 650 1344]]
-   io_gan
71.10%, 68.84%, 72.94%, 70.83%
[[1408  509]
 [ 621 1372]]
-   inner_dm
72.93%, 67.12%, 75.81%, 71.20%
[[1574  426]
 [ 654 1335]]
-   outer_dm
57.24%, 67.75%, 56.48%, 61.61%
[[ 903 1041]
 [ 643 1351]]
-   io_dm
63.43%, 68.24%, 62.47%, 65.23%
[[1153  816]
 [ 632 1358]]
-   inner_all
66.67%, 65.15%, 67.73%, 66.41%
[[1329  619]
 [ 695 1299]]
-   outer_all
67.30%, 66.85%, 67.94%, 67.39%
[[1320  628]
 [ 660 1331]]
-   all
67.24%, 67.34%, 68.23%, 67.78%
[[1277  625]
 [ 651 1342]]
